In [1]:
# Adjust Python path within the notebook
import sys
project_root = '/Users/thangnguyen/Documents/GitHub/project-1-individual-knam2609'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import scripts

In [2]:
from functools import reduce
from pyspark.sql.functions import col, sum as sum_, unix_timestamp

In [3]:
# Create SparkSession
spark = scripts.clean_base.create_spark_session()

24/08/27 15:27:30 WARN Utils: Your hostname, THANGs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.86.89.69 instead (on interface en0)
24/08/27 15:27:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 15:27:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/27 15:27:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
def count_null_values(df):
    """Count null values in each column of dataframe df"""
    null_counts = df.select([sum_(col(c).isNull().cast("int")).alias(c) for c in df.columns])
    print("Null values of each column:")
    null_counts.show(vertical=True)


In [5]:
# Inspect yellow taxi's data
yellow_files = scripts.clean_base.list_files_in_directory("../data/landing/yellow/")
yellow_dfs = [spark.read.parquet(file) for file in yellow_files]

In [6]:
yellow_dfs[0].printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [7]:
for df in yellow_dfs:
    print(yellow_files[yellow_dfs.index(df)])
    print(f"Number of rows: {df.count()}\n")
    print("Number of trips of different vendors:")
    df.groupBy("VendorID").count().show()
    # Calculate trip time in minutes
    df = df.withColumn("trip_time", (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60)
    # Calculate fare per miles
    df = df.withColumn("fare_per_miles", col("total_amount") / col("trip_distance"))
    print(df.filter(df["trip_time"] < 0).count())
    print("Min-Max values of each continous columns:")
    scripts.manipulate_data.find_min_max_df(df, scripts.clean_yellow.COLUMNS).show()
    print("Number of outliers:")
    print("For fare_per_miles:") 
    print(scripts.manipulate_data.detect_outliers(df, "fare_per_miles", 0.1, 0.9)[0].count())
    print("For trip_distance:")
    print(scripts.manipulate_data.detect_outliers(df, "trip_distance", 0.1, 0.9)[0].count())
    print("For trip_time:")
    print(scripts.manipulate_data.detect_outliers(df, "trip_time", 0.1, 0.9)[0].count())
    count_null_values(df)
    condition = reduce(lambda a, b: a | b, (col(c).isNull() for c in df.columns))
    df_with_null = df.filter(condition)
    print(f"Number of rows with null: {df_with_null.count()}\n")
    print("Vendors with null records:")
    df_with_null.groupBy("VendorID").count().show()

../data/landing/yellow/2023-08.parquet
Number of rows: 2824209

Number of trips of different vendors:
+--------+-------+
|VendorID|  count|
+--------+-------+
|       1| 716980|
|       2|2106608|
|       6|    621|
+--------+-------+

146
Min-Max values of each continous columns:
+--------------------+------------------+-----------------+
|              Column|               Min|              Max|
+--------------------+------------------+-----------------+
|     passenger_count|               0.0|              9.0|
|       trip_distance|               0.0|        345729.44|
|         fare_amount|            -900.0|           1375.0|
|               extra|              -7.5|            14.25|
|             mta_tax|              -0.5|              4.0|
|          tip_amount|             -90.7|            353.5|
|        tolls_amount|             -54.0|           170.75|
|improvement_surch...|              -1.0|              1.0|
|        total_amount|            -901.0|          1435.19

lower: -14.02687640153014, upper: 39.15898298377523
39510
For trip_distance:
lower: -11.770000000000001, upper: 21.43
19322
For trip_time:
lower: -35.78333333333333, upper: 72.21666666666667
15276
Null values of each column:
-RECORD 0----------------------
 VendorID              | 0     
 tpep_pickup_datetime  | 0     
 tpep_dropoff_datetime | 0     
 passenger_count       | 87886 
 trip_distance         | 0     
 RatecodeID            | 87886 
 store_and_fwd_flag    | 87886 
 PULocationID          | 0     
 DOLocationID          | 0     
 payment_type          | 0     
 fare_amount           | 0     
 extra                 | 0     
 mta_tax               | 0     
 tip_amount            | 0     
 tolls_amount          | 0     
 improvement_surcharge | 0     
 total_amount          | 0     
 congestion_surcharge  | 87886 
 Airport_fee           | 87886 
 trip_time             | 0     
 fare_per_miles        | 57429 

Number of rows with null: 126767

Vendors with null records:
+--------

24/08/27 15:27:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


-RECORD 0-----------------------
 VendorID              | 0      
 tpep_pickup_datetime  | 0      
 tpep_dropoff_datetime | 0      
 passenger_count       | 132675 
 trip_distance         | 0      
 RatecodeID            | 132675 
 store_and_fwd_flag    | 132675 
 PULocationID          | 0      
 DOLocationID          | 0      
 payment_type          | 0      
 fare_amount           | 0      
 extra                 | 0      
 mta_tax               | 0      
 tip_amount            | 0      
 tolls_amount          | 0      
 improvement_surcharge | 0      
 total_amount          | 0      
 congestion_surcharge  | 132675 
 Airport_fee           | 132675 
 trip_time             | 0      
 fare_per_miles        | 102583 

Number of rows with null: 173321

Vendors with null records:
+--------+-----+
|VendorID|count|
+--------+-----+
|       1|94464|
|       2|78403|
|       6|  454|
+--------+-----+

../data/landing/yellow/2023-10.parquet
Number of rows: 3522285

Number of trips of different

-RECORD 0-----------------------
 VendorID              | 0      
 tpep_pickup_datetime  | 0      
 tpep_dropoff_datetime | 0      
 passenger_count       | 154929 
 trip_distance         | 0      
 RatecodeID            | 154929 
 store_and_fwd_flag    | 154929 
 PULocationID          | 0      
 DOLocationID          | 0      
 payment_type          | 0      
 fare_amount           | 0      
 extra                 | 0      
 mta_tax               | 0      
 tip_amount            | 0      
 tolls_amount          | 0      
 improvement_surcharge | 0      
 total_amount          | 0      
 congestion_surcharge  | 154929 
 Airport_fee           | 154929 
 trip_time             | 0      
 fare_per_miles        | 121686 

Number of rows with null: 199721

Vendors with null records:
+--------+------+
|VendorID| count|
+--------+------+
|       1|110329|
|       2| 88890|
|       6|   502|
+--------+------+

../data/landing/yellow/2023-09.parquet
Number of rows: 2846722

Number of trips of di

...

lower: -43.50833333333333, upper: 86.42500000000001
10125
Null values of each column:
-RECORD 0----------------------
 VendorID              | 0     
 tpep_pickup_datetime  | 0     
 tpep_dropoff_datetime | 0     
 passenger_count       | 99887 
 trip_distance         | 0     
 RatecodeID            | 99887 
 store_and_fwd_flag    | 99887 
 PULocationID          | 0     
 DOLocationID          | 0     
 payment_type          | 0     
 fare_amount           | 0     
 extra                 | 0     
 mta_tax               | 0     
 tip_amount            | 0     
 tolls_amount          | 0     
 improvement_surcharge | 0     
 total_amount          | 0     
 congestion_surcharge  | 99887 
 Airport_fee           | 99887 
 trip_time             | 0     
 fare_per_miles        | 47472 

Number of rows with null: 141492

Vendors with null records:
+--------+------+
|VendorID| count|
+--------+------+
|       1| 38750|
|       6|  1113|
|       2|101629|
+--------+------+

../data/landing/yello

lower: -17.330206378986865, upper: 43.99812382739212
32024
For trip_distance:


lower: -15.590000000000003, upper: 27.85
5739
For trip_time:


lower: -33.141666666666666, upper: 66.39166666666667
22986
Null values of each column:


-RECORD 0----------------------
 VendorID              | 0     
 tpep_pickup_datetime  | 0     
 tpep_dropoff_datetime | 0     
 passenger_count       | 85086 
 trip_distance         | 0     
 RatecodeID            | 85086 
 store_and_fwd_flag    | 85086 
 PULocationID          | 0     
 DOLocationID          | 0     
 payment_type          | 0     
 fare_amount           | 0     
 extra                 | 0     
 mta_tax               | 0     
 tip_amount            | 0     
 tolls_amount          | 0     
 improvement_surcharge | 0     
 total_amount          | 0     
 congestion_surcharge  | 85086 
 Airport_fee           | 85086 
 trip_time             | 0     
 fare_per_miles        | 49790 

Number of rows with null: 123909

Vendors with null records:
+--------+-----+
|VendorID|count|
+--------+-----+
|       1|38757|
|       6|  724|
|       2|84428|
+--------+-----+



In [8]:
yellow_dfs[0].printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



It is pretty clear the values are missing completely at randomn => I decide to drop all the null records. 
Furthermore, I am seeing some abnormally high distance, fare here (> 300k), together with abnormally long trip time so let's have a look at the HVFHV data to see if it also has these problems. Also there are some weird negative and 0 values that should be positive.

In [9]:
# Inspect HVFHV's data
hvfhv_files = scripts.clean_base.list_files_in_directory("../data/landing/high_volume/")
hvfhv_dfs = [spark.read.parquet(file) for file in hvfhv_files]

In [10]:
hvfhv_dfs[0].printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_

In [11]:
for df in hvfhv_dfs:
    print(hvfhv_files[hvfhv_dfs.index(df)])
    print(f"Number of rows: {df.count()}\n")
    print("Number of trips of different brands:")
    df.groupBy("hvfhs_license_num").count().show()
    df = scripts.clean_base.rename_column(df, ["trip_miles"], ["trip_distance"])
    df = df.withColumn("total_amount", (col("base_passenger_fare") + col("tolls") + col("bcf") + col("sales_tax") + col("congestion_surcharge") + col("airport_fee") + col("tips")))
    # Calculate the waiting time in minutes
    df = df.withColumn("waiting_time", (unix_timestamp(col("on_scene_datetime")) - unix_timestamp(col("request_datetime"))) / 60)
    # Turn trip_time to minutes
    df = df.withColumn("trip_time", (col("trip_time") / 60))
    # Calculate fare per miles
    df = df.withColumn("fare_per_miles", (col("total_amount") / col("trip_distance")))
    print("Min-Max values of each continous columns:")
    scripts.manipulate_data.find_min_max_df(df, scripts.clean_high_volume.COLUMNS).show()
    print("Quantiles of fare_per_miles and Number of outliers:") 
    print("For fare_per_miles:") 
    print(scripts.manipulate_data.detect_outliers(df, "fare_per_miles", 0.1, 0.9)[0].count())
    print("For trip_distance:")
    print(scripts.manipulate_data.detect_outliers(df, "trip_distance", 0.1, 0.9)[0].count())
    print("For trip_time:")
    print(scripts.manipulate_data.detect_outliers(df, "trip_time", 0.1, 0.9)[0].count())
    count_null_values(df)
    condition = reduce(lambda a, b: a | b, (col(c).isNull() for c in df.columns))
    df_with_null = df.filter(condition)
    print(f"Number of rows with null: {df_with_null.count()}\n")
    print("Brand with null records:")
    df_with_null.groupBy("hvfhs_license_num").count().show()


../data/landing/high_volume/2023-08.parquet
Number of rows: 18322150

Number of trips of different brands:
+-----------------+--------+
|hvfhs_license_num|   count|
+-----------------+--------+
|           HV0005| 5178981|
|           HV0003|13143169|
+-----------------+--------+

Min-Max values of each continous columns:


+--------------------+-------------------+------------------+
|              Column|                Min|               Max|
+--------------------+-------------------+------------------+
|       trip_distance|                0.0|            483.91|
|           trip_time|                0.0| 598.0833333333334|
| base_passenger_fare|             -47.86|           2398.27|
|               tolls|                0.0|            105.12|
|                 bcf|                0.0|             66.42|
|           sales_tax|                0.0|            214.35|
|congestion_surcharge|                0.0|              8.25|
|         airport_fee|                0.0|               6.9|
|                tips|                0.0|             202.4|
|          driver_pay|             -60.93|           2942.83|
|        total_amount|             -30.83|2695.9700000000003|
|        waiting_time|-277.76666666666665| 942.3666666666667|
|      fare_per_miles|-13.025210084033615|           14752.0|
+-------

lower: -9.53908913352273, upper: 26.312544389204547
224047
For trip_distance:


lower: -15.178499999999996, upper: 28.097499999999997
171676
For trip_time:


24/08/27 15:28:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


...

lower: -14.216500000000003, upper: 26.707500000000003
204198
For trip_time:


lower: -37.016666666666666, upper: 79.25
88373
Null values of each column:


-RECORD 0-----------------------
 hvfhs_license_num    | 0       
 dispatching_base_num | 0       
 originating_base_num | 5388602 
 request_datetime     | 0       
 on_scene_datetime    | 5388602 
 pickup_datetime      | 0       
 dropoff_datetime     | 0       
 PULocationID         | 0       
 DOLocationID         | 0       
 trip_distance        | 0       
 trip_time            | 0       
 base_passenger_fare  | 0       
 tolls                | 0       
 bcf                  | 0       
 sales_tax            | 0       
 congestion_surcharge | 0       
 airport_fee          | 0       
 tips                 | 0       
 driver_pay           | 0       
 shared_request_flag  | 0       
 shared_match_flag    | 0       
 access_a_ride_flag   | 0       
 wav_request_flag     | 0       
 wav_match_flag       | 0       
 total_amount         | 0       
 waiting_time         | 5388602 
 fare_per_miles       | 3193    



Number of rows with null: 5391024

Brand with null records:


+-----------------+-------+
|hvfhs_license_num|  count|
+-----------------+-------+
|           HV0005|5388622|
|           HV0003|   2402|
+-----------------+-------+



In [12]:
hvfhv_dfs[0].printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_

Not so much with HVFHV's data. The values are mostly missing on records of Lyft, the missing data of Uber is only due to the trip_distance being 0 so the fare_per_miles is undefined, the values missing are of on_scene_datetime which I genuinely need for my waiting-time analysis + we have a lot of Uber's data => I am dropping all records of Lyft and focus on analyzing Uber's records only.
About the abnormally high distance, fare and trip time, HVFHV doesn't have those problems here but it still has abornormally high fare_per_miles and  weird negative and 0 values => I decided to drop the negative and non-positive values depending on the features, drop records which have fare_per_miles outside the 2 quantile bounds because the bounds are resonable => this will take care of the high total amount problem. For the high distance and trip time, I choose to drop any distance higher than 500 and any time longer than 750 minutes as most of the maximum values of Uber are around this time. 